In [1]:
import pandas as pd
import seaborn as sns

In [2]:
df = pd.read_csv('listings.csv',delimiter=',',low_memory=False)

In [3]:
df.shape

(8459, 106)

In [16]:
df_2 = df[['host_since','host_response_time','host_response_rate','host_total_listings_count'
           ,'host_verifications','host_has_profile_pic','host_identity_verified',
           'neighbourhood_cleansed','zipcode','latitude','longitude',
           'room_type','bathrooms','bedrooms','beds','bed_type','amenities','price',
           'security_deposit','cleaning_fee','extra_people','minimum_nights',
           'calendar_updated','availability_30','availability_90','cancellation_policy',
           'reviews_per_month','review_scores_rating']]

In [17]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8459 entries, 0 to 8458
Data columns (total 28 columns):
host_since                   8459 non-null object
host_response_time           6505 non-null object
host_response_rate           6505 non-null object
host_total_listings_count    8459 non-null int64
host_verifications           8459 non-null object
host_has_profile_pic         8459 non-null object
host_identity_verified       8459 non-null object
neighbourhood_cleansed       8459 non-null object
zipcode                      8383 non-null float64
latitude                     8459 non-null float64
longitude                    8459 non-null float64
room_type                    8459 non-null object
bathrooms                    8456 non-null float64
bedrooms                     8459 non-null int64
beds                         8458 non-null float64
bed_type                     8459 non-null object
amenities                    8459 non-null object
price                        8459 non-nu

In [18]:
df_2.review_scores_rating.value_counts()

100.0    1808
98.0      949
99.0      834
97.0      764
96.0      618
95.0      486
93.0      379
94.0      314
90.0      233
92.0      193
91.0      158
80.0      145
89.0       97
88.0       86
87.0       83
85.0       51
86.0       35
60.0       34
83.0       26
84.0       25
73.0       16
70.0       16
82.0       10
76.0        9
81.0        9
75.0        7
20.0        6
40.0        6
79.0        6
77.0        6
78.0        5
67.0        5
65.0        4
71.0        4
74.0        3
69.0        2
53.0        1
72.0        1
50.0        1
68.0        1
Name: review_scores_rating, dtype: int64

In [19]:
df_geo = df_2[["latitude","longitude"]]

In [20]:
df_geo.head()

,latitude,longitude
0,47.610819,-122.290816
1,47.687801,-122.313427
2,47.523980,-122.359891
3,47.654109,-122.337605
4,47.550620,-122.320135


In [ ]:
def get_walkscore_url(address, city, zip_code, lat, lon):
    """
    Construct url for Walkscore api call
    Input: address, city, and zip_code as strings; lat/lon coords as float
    Output: prepared url to request walkscore for address
    """
    api_key = '466d1cb991e8a99345b049d505c6a4a7'
    base_url = 'http://api.walkscore.com/score?format=json'
    mid_url = 'transit=1&bike=1'
    address = 'address=' + '%'.join(address.split())
    address = '%20'.join([address, city, 'WA', zip_code])
    lat = f'lat={lat}'
    lon = f'lon={lon}'
    api_key = f'wsapikey={api_key}'
    url = '&'.join([base_url, address, lat, lon, mid_url, api_key])
    return url


def get_walkscores(row):
    """
    Makes api call to Walkscore and extracts bike, walk, and transit scores
    Input: dataframe row containing required fields
    Output: list containing bike, walk, and transit scores (or nan, if failure)
    """
    fields = ['address1', 'city', 'zip_code', 'latitude', 'longitude']
    address, city, zip_code, lat, lon = row[fields]
    url = get_walkscore_url(address, city, zip_code, lat, lon)
    
    try:
        r = requests.get(url)
        response = json.loads(r.text)

        bike_score = response['bike']['score']
        walk_score = response['walkscore']
        transit_score = response['transit']['score']
    except:
        return (np.nan, np.nan, np.nan)
    return [bike_score, walk_score, transit_score]

In [ ]:
def get_demo_data(path, license_filename, demo_filename):
    """
    Scrapes zip code based demographic data from washington.hometownlocator.com
    for all zip codes containing a dispensary found in WSLCB license data
    Input: relative path to raw data directory, license data filename, 
    demographics data filename
    Output: saves demographic dataset to csv in raw data directory
    """
    license_filepath = path + license_filename
    demo_filepath = path + demo_filename

    if os.path.isfile(demo_filepath):
        overwrite = input(
            '''Demographics file already exists. Scrape data again? y/n\n 
            Note: this could take several minutes.''')
        if overwrite.lower() != 'y':
            return
    
    license_data = pd.read_excel(license_filepath, sheet_name=2, header=0)
    zips = license_data['ZipCode'].astype(str).str[:5].unique()
    demographics = pd.DataFrame()
    
    print("Beginning scrape...")
    for zip_code in sorted(zips):
        url = f'https://washington.hometownlocator.com/zip-codes/data,zipcode,{zip_code}.cfm'
        r = requests.get(url)
        if 'table' in r.text:
            df0, df1 = pd.read_html(url, index_col=0)[:2]
            df0.columns = [str(zip_code)]
            df1.columns = [str(zip_code)]
            df = pd.concat([df0, df1], axis=0).T.dropna(axis=1)
            df.drop(['INCOME', 'HOUSEHOLDS'], axis=1, inplace=True)
            demographics = pd.concat([demographics, df])
            print('Scraped {}/{} zips. Latest: {}'
                  .format(len(demographics), len(zips), zip_code), end='\r')
            sys.stdout.flush()
        else:
            print(f'\nNo data found for {zip_code}')
        
    demographics.to_csv(demo_filepath)
    print('Scraped data written to {}'.format(demo_filepath))
    return